In [ ]:
# install some libraries if not installed already
!pip install scikit-image 
!pip install imgaug
!pip install pandas
!pip install pydicom

In [ ]:
# import some libraries we will use
import os, sys, glob, csv
import pydicom

import matplotlib.pyplot as plt
import numpy as np

from skimage import io

In [ ]:
# MNIST handwritten digits

In [ ]:
# directories for training and testing sets 
mnist_base_dir_tr = '/data/user/mnist_png/mnist_png/training/'
mnist_base_dir_te = '/data/user/mnist_png/mnist_png/testing/'

In [ ]:
# list '1' images
mnist_ones_tr = glob.glob(os.path.join(mnist_base_dir_tr, '1', '*.png'))
mnist_ones_te = glob.glob(os.path.join(mnist_base_dir_te, '1', '*.png'))

In [ ]:
# display 10 examples
for idx, imf in enumerate(mnist_ones_tr):
    plt.figure()
    plt.imshow(io.imread(imf), cmap='gray')
    
    if idx >=10:
        break

In [ ]:
# Task 1: try visualize test cases and other digits categories

In [ ]:
# Task 2: try to randomize which 10 images you visualize

In [ ]:
# Chest x-ray pneumonia detection dataset

In [ ]:
# chest x-ray root directory (that you downloaded from week-01)
chestpnx_dir = '/data/user/pneumonia_detection_data/'
stage_num = '1'
test_only_part = True

In [ ]:
# import custom-written functions
import image_bbox_loader
import box_utils

In [ ]:
# load images, bounding-boxes, labels using the custom-written functions
image_bbox_label_list = image_bbox_loader.load_image_bbox_label_list(chestpnx_dir,
                                                                     stage_num, test_only_part)

In [ ]:
# show one example image
for imf, boxes, labels in image_bbox_label_list:
    img = pydicom.dcmread(imf).pixel_array
    plt.imshow(img, cmap='gray')
    break

In [ ]:
# import imgaug package for visuzlizing bounding-boxes
from imgaug import augmenters as iaa
import imgaug as ia

In [ ]:
# show one example image with bounding boxes
for imf, boxes, labels in image_bbox_label_list:
    img = pydicom.dcmread(imf).pixel_array
    img2 =  np.stack((img,)*3, axis=-1)
    
    for boxi_ in boxes: # batch_size=1
        
        boxi = box_utils.center_form_to_corner_form(
               box_utils.min_wh_form_to_center_form(boxi_)).numpy()
        
        bbi = ia.BoundingBox(x1=boxi[0],
                             x2=boxi[2],
                             y1=boxi[1],
                             y2=boxi[3])
        img2 = bbi.draw_on_image(img2, thickness=6)
    
    plt.imshow(img2)
    break

In [ ]:
# Task 3: try to visualize 10 random examples of chest x-ray with pneumonia